In [ ]:
import argparse, os
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
os.getenv("USERNAME") == "jmendillo"


# Lookup table for counties
LOOKUP_COUNTY = pd.DataFrame({
           "county2": ["AL", "CC", "MA", "NA", "SF", "SM", "SC", "SL", "SN"],
    "county_census": ["001", "013", "041", "055", "075", "081", "085", "095", "097"],
    "county_name": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma"]
}) 

AIR_BASINS = pd.DataFrame({
    "air_basin": ["Sonoma (SF)", "Solano(SF)", "Santa Clara (SF)", "Sonoma (NC)","Solano (SV)","Marin (SF)","Napa (SF)", "San Franscisco (SF)","San Mateo (SF)","Contra Costa (SF)","Alameda (SF)"]
})

def half_split(strng, sep, pos):
    strng = strng.split(sep)
    return sep.join(strng[:pos]), sep.join(strng[pos:])


In [ ]:
#set working directory
BASE_DIR="C:/Users/Jmendillo/Box/Plan Bay Area 2050+/EIR/Quantitative Analyses/MSAT Emissions"
RAW_EMISSION_RATES_DIR = os.path.join(BASE_DIR, "CT-EMFAC2021 Output/Default Fleet Mix").replace("\\","/")

onlyfiles = [f for f in listdir(RAW_EMISSION_RATES_DIR) if isfile(join(RAW_EMISSION_RATES_DIR, f))]
print(onlyfiles)
x = 'Alameda (SF) - 2023 - Annual.EF'
filename = os.path.join(BASE_DIR, f"CT-EMFAC2021 Output/Default Fleet Mix/{x}").replace("\\","/")
f=open(filename)
file_lines = f.readlines()
pollutants_list_temp = file_lines[26:48]
pollutants_list_for_df = []
stripped_pollutants_list = [s.strip() for s in pollutants_list_temp]
stripped_pollutants_list = [s.split("\t ") for s in stripped_pollutants_list]
for list in stripped_pollutants_list: 
    list = [s.strip() for s in list]
    list.append("test")
    pollutants_list_for_df.append(list)
print(pollutants_list_for_df)

print(len)
    

In [7]:
#create e2025 MSAT emission rates dataframe from raw text output (2023 and 2050)

frames =[]

for root, dirs, files in os.walk(f"{BASE_DIR}/CT-EMFAC2021 Output/Default Fleet Mix"):
    for file in files: 

        if file.endswith(".EF"):
            filename = os.path.join(root, file)
        
        county = file.split(" - ",2)[0]
        year = file.split(" - ",2)[1]
        pollutants_list_for_df = []
        with open(filename) as f:
            f=open(filename)
            file_lines = f.readlines()
            pollutants_list_temp = file_lines[26:48]

            #clean up string outputs and add to the empty list
            
            stripped_pollutants_list = [s.strip() for s in pollutants_list_temp]
            stripped_pollutants_list = [s.split("\t ") for s in stripped_pollutants_list]
            for list in stripped_pollutants_list: 
                list = [s.strip() for s in list]
                #add county and year 
                # list.append(f"{county}")
                # list.append(f"{year}")
                #add to empty list
                pollutants_list_for_df.append(list)
            df = pd.DataFrame(pollutants_list_for_df,columns=['Pollutant Name', '<= 5 mph', '10 mph', '15 mph', '20 mph','25 mph', '30 mph', '35 mph', '40 mph', '45 mph', '50 mph','55 mph', '60 mph', '65 mph', '70 mph', '75 mph'])
            df=df.transpose()
            df.columns = df.iloc[0]
            df = df.drop(df.index[0])
            df = df.drop('Pollutant Name',axis=1)
            df.insert(0, 'County', county)
            df.insert(1, 'Year', year)
            frames.append(df)
            
print(type(frames[0]))
result = pd.concat(frames)
# result.rename(columns={'Pollutant Name': "Speed Bin"})
result

<class 'pandas.core.frame.DataFrame'>


Pollutant Name,County,Year,PM2.5,PM10,NOx,CO,HC,TOG,ROG,"1,3-Butadiene",...,Diesel PM,Ethylbenzene,Formaldehyde,Naphthalene,POM,DEOG,CO2,N2O,CH4,BC
<= 5 mph,Alameda (SF),2023,0.010266,0.011061,0.667065,1.742195,0.161725,0.177821,0.117473,0.000475,...,0.002397,0.001695,0.004348,0.000484,0.000126,0.013776,883.411213,0.037191,0.039422,0.002292
10 mph,Alameda (SF),2023,0.006939,0.007462,0.507889,1.516412,0.105080,0.115720,0.076238,0.000308,...,0.002078,0.001092,0.003006,0.000312,0.000084,0.012256,724.424026,0.035525,0.026859,0.001501
15 mph,Alameda (SF),2023,0.004811,0.005168,0.367835,1.331350,0.069837,0.076838,0.051003,0.000209,...,0.001578,0.000739,0.002096,0.000211,0.000058,0.007736,592.202492,0.029409,0.017909,0.001038
20 mph,Alameda (SF),2023,0.003501,0.003757,0.299580,1.191943,0.049710,0.054638,0.036187,0.000150,...,0.001230,0.000526,0.001535,0.000150,0.000041,0.005359,498.928228,0.026038,0.013175,0.000754
25 mph,Alameda (SF),2023,0.002708,0.002903,0.248672,1.079898,0.037723,0.041495,0.027376,0.000113,...,0.001030,0.000395,0.001191,0.000112,0.000032,0.004294,431.550264,0.023569,0.010375,0.000575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55 mph,Sonoma (SF),2050,0.000742,0.000788,0.033000,0.368184,0.006310,0.006907,0.004516,0.000021,...,0.000475,0.000067,0.000202,0.000018,0.000005,0.000676,233.994072,0.008903,0.002153,0.000103
60 mph,Sonoma (SF),2050,0.000866,0.000918,0.036731,0.346472,0.006654,0.007272,0.004780,0.000022,...,0.000584,0.000072,0.000212,0.000019,0.000005,0.000717,243.161456,0.009141,0.002230,0.000110
65 mph,Sonoma (SF),2050,0.001031,0.001093,0.043370,0.327782,0.007358,0.008031,0.005338,0.000025,...,0.000711,0.000081,0.000229,0.000021,0.000006,0.000776,251.665873,0.009585,0.002379,0.000121
70 mph,Sonoma (SF),2050,0.001070,0.001135,0.044182,0.320514,0.007842,0.008541,0.005706,0.000025,...,0.000711,0.000081,0.000229,0.000021,0.000006,0.000776,254.410389,0.009560,0.002486,0.000121


In [11]:
result.to_csv('C:/Users/Jmendillo/Box/Plan Bay Area 2050+/EIR/Quantitative Analyses/MSAT Emissions/MSAT Emission Rates.csv')